In [25]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from train import *
# from test import *
from portraitNet import ProtraitNet
import torchvision.models.mobilenetv2
from easydict import EasyDict as edict
from data.datasets import *

In [26]:
from tqdm import tqdm
import torch
import torch.nn as nn
import numpy as np
from loss import *
from torch.autograd import Variable
import copy
from metrice import *

# -----model hyperparameters-------
# temperature in consistency constraint loss
temperature = 1
# the weight of consistency constraint loss
alpha = 2


def test(dataloader, model , device = "cpu"):
    # switch to eval mode
    model.eval()# switch to eval mode
    
    loss_Focalloss = FocalLoss(gamma=2)
    loss_Softmax = nn.CrossEntropyLoss(ignore_index=255)
    losses = []
    
    loader = tqdm(dataloader)
    iou = 0

    with torch.no_grad():
        # for idx, (img, mask, mask_boundary) in enumerate(loader):
        for i, (input_ori, input, edge, mask) in enumerate(loader):
            # Send to device
            input_ori = input_ori.to(device)
            input = input.to(device)
            mask = mask.to(device)
            edge = edge.to(device)

            # load input to model
            output_mask_texture, output_edge_texture = model(input)
            output_mask_deformation, output_edge_deformation = model(input_ori)

            # mask loss and edge loss
            loss_mask_texture = loss_Softmax(output_mask_texture, mask)
            loss_edge_texture = loss_Focalloss(output_edge_texture, edge) * 0.1
            loss_mask_deformation = loss_Softmax(output_mask_deformation, mask)
            loss_edge_deformation = loss_Focalloss(output_edge_deformation, edge) * 0.1

            # consistency constraint loss: KL distance
            loss_kl_mask = loss_KL(output_mask_texture, Variable(output_mask_deformation.data, requires_grad=False),
                                temperature) * alpha
            loss_kl_edge = loss_KL(output_edge_texture, Variable(output_edge_deformation.data, requires_grad=False),
                                temperature) * alpha * 0.1

            #total loss
            # loss = loss_mask_texture + loss_edge_texture + loss_mask_deformation + loss_edge_deformation + loss_kl_mask + loss_kl_edge
            # loss = loss_mask + loss_mask_ori + loss_kl_mask + loss_edge
            loss = loss_mask_texture + loss_mask_deformation + loss_kl_mask + loss_edge_texture
                
            losses.append(loss.item())

            softmax = nn.Softmax(dim=1)
            prob = softmax(output_mask_texture)[0,1,:,:]
            pred = prob.data.cpu().numpy()
            pred[pred>0.5] = 1
            pred[pred<=0.5] = 0
            iou += calcIOU(pred, mask[0].data.cpu().numpy())

    return sum(losses)/len(losses),  iou / len(loader)

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# -------use official dataset definition--------------
exp_args = edict()
exp_args.istrain = True
exp_args.task = 'seg'
exp_args.datasetlist =['EG1800']
exp_args.data_root = './datasets/'
exp_args.file_root = './datasets/EG1800/'

exp_args.input_height = 224
exp_args.input_width = 224


exp_args.prior_prob = 0.5 # the probability to set empty prior channel

exp_args.edgeRatio = 0.1 # the weight of boundary auxiliary loss
# exp_args.stability = True
exp_args.temperature = 1 # the temperature in consistency constraint loss, default=1
exp_args.alpha = 2 # the weight of consistency constraint loss, default=2
############################
exp_args.padding_color=128 # input normalization parameters
exp_args.img_scale = 1
exp_args.img_mean = [103.94, 116.78, 123.68] # BGR order, image mean
exp_args.img_val = [0.017, 0.017, 0.017] # BGR order, image val, default=[1/0.017, 1/0.017, 1/0.017]
##########################
exp_args.init = False # whether to use pretrained model to init portraitnet
exp_args.resume = False # whether to continue training

# set training dataset
exp_args.learning_rate = 1e-3
exp_args.momentum = 0.9
exp_args.weight_decay = 5e-4
exp_args.batch_size = 1
#######################下面没什么用
exp_args.addEdge = True
exp_args.stability = True
exp_args.use_kl = True
exp_args.useUpsample = False
exp_args.useDeconvGroup = False
exp_args.video = False
exp_args.istrain = True

dataset_train = Human(exp_args)
dataLoader_train = torch.utils.data.DataLoader(dataset_train, batch_size=64,
                                               shuffle=True)

# set testing dataset
exp_args.istrain = False
dataset_test = Human(exp_args)
dataLoader_test = torch.utils.data.DataLoader(dataset_test, batch_size=64,
                                              shuffle=False)


In [28]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"
model_best = ProtraitNet(n_class=2).to(device)
model_best.load_state_dict(torch.load("./myPortraitNet_usepretrained.ckpt", map_location=torch.device('cpu')))
test_loss,test_iou = test(dataLoader_test,model_best,device)
print(f" test_loss = {test_loss:.5f}, test_iou = {test_iou:.5f}")

  0%|          | 0/5 [00:00<?, ?it/s]/Users/koala/Desktop/cuhksz2023sem1/cv/portraitNet/myPortraitNet/loss.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
/Users/koala/miniforge3/envs/pytorch/lib/python3.10/site-packages/torch/nn/functional.py:2943: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
100%|██████████| 5/5 [00:34<00:00,  6.88s/it]

 test_loss = 0.12371, test_iou = 0.97246
